# SubGraph builder

Ini discussion with the Pacific group there was interest in what can extracted from OIH and fed back to SPREP.  Part of this was driven by a desire to show benefit from the collaboration.  


### Imports


In [1]:
import dask
import boto3
import pandas as pd
import json, io
import pathlib
import s3fs
import kglab
from rdflib import Graph  #, plugin
from pyld import jsonld
from spacy import displacy
import spacy
import en_core_web_lg  # intersting need for import here...

### Set up Dask client

In [2]:
from dask.distributed import Client    #, progress
client = Client(threads_per_worker=5, n_workers=4)
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 20,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:39505,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:37875,Total threads: 5
Dashboard: http://127.0.0.1:34603/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:46233,


### File Access

In [3]:
# Set up the S3 File system
# oss = s3fs.S3FileSystem(
#     anon=True,
#     client_kwargs = {"endpoint_url":"https://oss.geodex.org"}
# )

## Access controlled s3
session = boto3.Session(profile_name='default' ,   region_name="us-east-1")
s3 = session.client('s3')  # needed later for listing objects
s3r = session.resource('s3')
oss = s3fs.S3FileSystem( profile="default")


## Manual code access
# ACCESS_CODE = getpass.getpass()
# SECRET_CODE = getpass.getpass()

# oss = s3fs.S3FileSystem(
#     anon=False,
#     key=ACCESS_CODE,
#     secret=SECRET_CODE,
#     client_kwargs = {"endpoint_url":"http://192.168.86.45:49159"}
# )

In [4]:
# [optional] List the directories we can work with later
# sumlist = oss.ls('gleaner/summoned')
sumlist = oss.ls('gleaner.oih/summoned/')
print(sumlist)

['gleaner.oih/summoned/aquadocs', 'gleaner.oih/summoned/edmerp', 'gleaner.oih/summoned/edmo', 'gleaner.oih/summoned/invemardocuments', 'gleaner.oih/summoned/invemarexpert', 'gleaner.oih/summoned/invemarinstitution', 'gleaner.oih/summoned/invemartraining', 'gleaner.oih/summoned/invemarvessel', 'gleaner.oih/summoned/marinetraining', 'gleaner.oih/summoned/obis', 'gleaner.oih/summoned/obps']


## Keywords

Some keywords of relevance to look for????

In [5]:
kws = ['pacific', 'fiji', 'new caldonia', 'Tuvalu']
targets = ['.*genom.*', '.*DNA.*', '.*RNA.*', '16S', '12S', '18S']

## Framing 

A little playground to test frames

In [6]:
frame1 =  {"@context":{"@vocab": "https://schema.org/"}, "@type": "Course", "@explicit": "true", "description": {}};

In [7]:
dg = './datagraphs/datagraph.json'
with open(dg,"r") as f:
    jld = json.loads(f.read().replace('\n',' '))
    framed = jsonld.frame(jld, frame1)

print(framed)


{'@context': {'@vocab': 'https://schema.org/'}, '@type': 'Course', 'description': "In this course you will get an introduction to the main tools and ideas in the data scientist's toolbox..."}


## SHACL

See if there are tests that can be done in SHACL to see if a data graph is related to the keywords

## Dask processing

### Define Delayed Definitions

In [8]:
# Simple JSON-LD framing inside Dask function
@dask.delayed()
def citation_frame(fn):
    # or preferably open in text mode and json.load from the file
    citationframe = {"@context":{"@vocab": "https://schema.org/"}, "@type": "Dataset", "@explicit": "true", "description": {}};
    #   citationframe = {"@context":{"@vocab": ctx}, "@type": "Dataset", "@explicit": "true", "citation": {}};
    with oss.open(fn, 'rb') as f:
        #return json.loads(f.read().replace('\n',' '))
        jld = json.loads(f.read().decode("utf-8", "ignore").replace('\n',' '))
        framed = jsonld.frame(jld, citationframe)
        return framed   # ['citation']

In [9]:
# Read object 
@dask.delayed()
def read_a_file(fn):
    # or preferably open in text mode and json.load from the file
    with oss.open(fn, 'rb') as f:
        #return json.loads(f.read().replace('\n',' '))
        # return json.loads(f.read().decode("utf-8", "ignore").replace('\n',' '))

        jld =  json.loads(f.read().decode("utf-8", "ignore").replace('\n',' '))
        desc = jld["description"]

        if desc is None:
            desc = ""
        kws = jld["keywords"]
        name = jld["name"]
        url = jld["url"]
        # gldf = gldf.append({'name':name, 'url':url, 'keywords':kws, 'description': desc}, ignore_index=True)

    # gldf = gldf.append({'name':name, 'url':url, 'keywords':kws, 'description': desc}, ignore_index=True)
    return [name, url, kws, desc]



### Process files


#### Load examples to pandas

In [10]:
%%time
#  loop load into dataframe

filenames = oss.ls('gleaner.oih/summoned/obis')
o = [read_a_file(f) for f in filenames]
results = dask.compute(*o)

# read the array of lists into the datagram
gldf = pd.DataFrame.from_records(results, columns=['name', 'url', "keywords", "description"])


CPU times: user 4.32 s, sys: 494 ms, total: 4.82 s
Wall time: 32.4 s


In [11]:
gldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4426 entries, 0 to 4425
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4415 non-null   object
 1   url          4426 non-null   object
 2   keywords     4408 non-null   object
 3   description  4426 non-null   object
dtypes: object(4)
memory usage: 138.4+ KB


In [12]:
gldf.head()

,name,url,keywords,description
0,Counts of breeding seabirds and herons at Ashm...,https://obis.org/dataset/c43d814e-5719-499d-a8...,"[Occurrence, Observation]",Ashmore Reef is situated on the edge of Sahul ...
1,Brazil Reef Biodiversity - ILTER Coastal Habit...,https://obis.org/dataset/c204e975-113e-4150-86...,"[ILTER, Coastal Habitats of Espírito Santo, Co...",The PELD (Long-term Program of Ecological Rese...
2,Netplankton North Adriatic-ALPE ADRIA Project,https://obis.org/dataset/8e1a7146-8eba-4c00-82...,"[Occurrence, Observation]",Netplankton data collected within the ALPEADRI...
3,Phytoplankton monitoring in the South Adriatic...,https://obis.org/dataset/36be7d35-bc96-4ba1-a2...,"[Occurrence, Observation]",Phytoplankton monitoring in the South Adriatic...
4,Zooplankton counts measured at the West-Hinder...,https://obis.org/dataset/b63b0f52-41c4-461d-84...,"[West-Hinder, Zooplankton species, Samplingevent]",This dataset is part of the historic monitorin...


In [13]:
text = gldf.at[1,'description']
kwtest = gldf.at[1,'keywords']

In [14]:
nlp = en_core_web_lg.load()
doc2 = nlp(text)
displacy_image = displacy.render(doc2, jupyter = True, style = 'ent')

#### Loop on all

Now lets loop on all of the data graphs and pull the descriptions.  We will then use spacy to identify entities and hold these for later use. 

In [19]:
%%time
# nlp = en_core_web_lg.load()
df2 = pd.DataFrame(columns=['name', 'label', 'text', 'url'])

for i in range(len(gldf)):
  doc3 = nlp( gldf.at[i,'description'])
  for entity in doc3.ents:
    df2 = df2.append({'name': gldf.at[i,'name'], 'label': entity.label_, 'text':entity.text, 'url': gldf.at[i,'url']}, ignore_index=True)

CPU times: user 3min 6s, sys: 1.94 s, total: 3min 8s
Wall time: 3min 5s


In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70603 entries, 0 to 70602
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    70603 non-null  object
 1   label   70603 non-null  object
 2   text    70603 non-null  object
 3   url     70603 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [21]:
df2.head(5)

,name,label,text,url
0,Counts of breeding seabirds and herons at Ashm...,PERSON,Ashmore Reef,https://obis.org/dataset/c43d814e-5719-499d-a8...
1,Counts of breeding seabirds and herons at Ashm...,GPE,Sahul Shelf,https://obis.org/dataset/c43d814e-5719-499d-a8...
2,Counts of breeding seabirds and herons at Ashm...,LOC,Kimberley,https://obis.org/dataset/c43d814e-5719-499d-a8...
3,Counts of breeding seabirds and herons at Ashm...,GPE,Australia,https://obis.org/dataset/c43d814e-5719-499d-a8...
4,Counts of breeding seabirds and herons at Ashm...,DATE,2010,https://obis.org/dataset/c43d814e-5719-499d-a8...


In [22]:
df2.to_csv("./output/obisNER.csv")


## Regex searches

https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/

targets = ['.*genom.*', '.*DNA.*', '.*RNA.*', '16S', '12S', '18S']

In [30]:
S = gldf['description']
print(S)

0       Ashmore Reef is situated on the edge of Sahul ...
1       The PELD (Long-term Program of Ecological Rese...
2       Netplankton data collected within the ALPEADRI...
3       Phytoplankton monitoring in the South Adriatic...
4       This dataset is part of the historic monitorin...
                              ...                        
4421    These inventories list marine species observat...
4422    Original provider:\nCécile Vincent, Centre d’É...
4423    Original provider:\nHappywhale\n\nDataset cred...
4424    Changes in turbidity, sedimentation and light ...
4425    The only Antarctic review about brachiopods ha...
Name: description, Length: 4426, dtype: object


In [31]:
S.str.count(r'(.genom.)').sum()

20

In [32]:
S.str.count(r'(.DNA.)').sum()

59

In [33]:
S.str.count(r'(.RNA.)').sum()

83

In [34]:
S.str.count(r'(16S)').sum()

72

In [35]:
S.str.count(r'(12S)').sum()

1

In [36]:
S.str.count(r'(18S)').sum()

24

# __________________________________

scratch notes below here

#### Citation Frame loop

In [7]:
%%time

fns = oss.ls('gleaner/summoned/lipdverse')
o = [citation_frame(f) for f in fns]

results = dask.compute(*o)  ## Wait..  can I just dask.compute(o)  ????
print(len(results))

697
CPU times: user 710 ms, sys: 79.3 ms, total: 790 ms
Wall time: 6.06 s


### Loop on results and load to graph

In [13]:
# You likely don't want to do this..   prints a lot of data
# for r in results:
#     print(r)
print(json.dumps(results[100]))

{"@context": {"@vocab": "https://schema.org/"}, "@id": "http://lipdverse.org/Temp12k/1_0_2/Topptjonna.Paus.2011.html", "@type": "Dataset", "description": "This dataset from Topptjonna (Europe>Northern Europe>Scandinavia>Norway) is derived from a LakeSediment archive, and includes data on OriginalSampleID, temperature, and ReliabIeYN1. The data are relevant to the time interval from 11824 to 6133 (Calibrated)."}


In [14]:
rnamespaces = {
    "schema":  "https://schema.org/",
    "shacl":   "http://www.w3.org/ns/shacl#" ,
}

rkg = kglab.KnowledgeGraph(
    name = "Schema.org shacl eval datagraph",
    base_uri = "https://example.org/id/",
    namespaces = rnamespaces,
)

g = rkg.rdf_graph()

for r in results:
    rkg.load_rdf_text(data=json.dumps(r), format="json-ld") 
        
   

In [15]:
sparql = """
PREFIX schema: <https://schema.org/>
SELECT *
  WHERE {
    ?s schema:description ?o .
  }
"""

df = rkg.query_as_df(sparql)

In [16]:
pdf = df.to_pandas()
# pdf.style.apply(change_color_group, axis=None)
pdf.info()
pdf.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697 entries, 0 to 696
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   s       697 non-null    object
 1   o       697 non-null    object
dtypes: object(2)
memory usage: 11.0+ KB


,s,o
0,<http://lipdverse.org/Temp12k/1_0_2/Wonderkrat...,This dataset from Wonderkrater (Africa>Souther...
1,<http://lipdverse.org/Temp12k/1_0_2/GeoB5844_2...,This dataset from GeoB5844_2 (Indian Ocean>Red...
2,<http://lipdverse.org/Temp12k/1_0_2/BuntesMoor...,This dataset from Buntes Moor (Europe>Western ...
3,<http://lipdverse.org/Temp12k/1_0_2/Mohawk.Web...,This dataset from Mohawk (North America>United...
4,<http://lipdverse.org/Temp12k/1_0_2/MD98_2195....,This dataset from MD98_2195 (Pacific Ocean>Wes...
5,<http://lipdverse.org/Temp12k/1_0_2/Churruca.C...,This dataset from Churruca (South America>Chil...
6,<http://lipdverse.org/Temp12k/1_0_2/LagunaChap...,This dataset from Laguna Chaplin (South Americ...
7,<http://lipdverse.org/Temp12k/1_0_2/Sfl4-1.Wil...,This dataset from sfl4-1 (North America>Greenl...
8,<http://lipdverse.org/Temp12k/1_0_2/HiddenLake...,This dataset from Hidden Lake CA (North Americ...
9,<http://lipdverse.org/Temp12k/1_0_2/Laihalampi...,This dataset from Laihalampi (Europe>Northern ...


In [17]:
# Optional export to parquet and or CSV
df.to_parquet("./output/frameresults.parquet")